In [1]:
import rasterio
from rasterio.windows import Window
import glob

# List all raster files
tiff_files = glob.glob(r"C:\Users\rubab\Desktop\GWE_codes\AncillaryWaterData\downloads_ancillary_downloadWaterData_PythonV3_2021\trans_output\reclassified\*.tif")

def merge_with_window(src_files, output_file, chunk_height=512, chunk_width=512):
    with rasterio.open(src_files[0]) as src:
        # Get dimensions of the output raster
        meta = src.meta.copy()
        meta.update(driver="GTiff", count=1)
        
        with rasterio.open(output_file, "w", **meta) as dest:
            # Iterate over windows
            for i in range(0, src.height, chunk_height):
                for j in range(0, src.width, chunk_width):
                    # Adjust window size at edges
                    height = min(chunk_height, src.height - i)
                    width = min(chunk_width, src.width - j)
                    window = Window(j, i, width, height)
                    
                    # Read and merge data from all rasters within the window
                    data = [
                        raster.read(1, window=window, boundless=True)
                        for raster in [rasterio.open(fp) for fp in src_files]
                    ]
                    
                    # Write merged data to the output file
                    dest.write(data[0], 1, window=window)

# Merge files
merge_with_window(tiff_files, "final_output.tif")
print("Final merged raster saved as 'final_output.tif'")


TypeError: expected string or bytes-like object, got 'NoneType'

In [11]:
import rasterio
import numpy as np
from rasterio.plot import show

def mask_raster(input_tiff, mask_tiff, output_tiff):
    # Open the input TIFF
    with rasterio.open(input_tiff) as src:
        input_data = src.read(1)  # Read the first band
        input_meta = src.meta.copy()
        
        # Open the mask TIFF
        with rasterio.open(mask_tiff) as mask_src:
            mask_data = mask_src.read(1)  # Read the first band of the mask
            
            # Ensure input and mask have the same shape
            if src.shape != mask_src.shape:
                raise ValueError("Input and mask TIFFs must have the same dimensions.")
            
            # Mask out regions: Set input_data to nodata where mask_data > 0
            masked_data = np.where(mask_data > 0, np.nan, input_data)
            
            # Update metadata for nodata
            input_meta.update({"nodata": np.nan})
            
            # Save the masked raster
            with rasterio.open(output_tiff, "w", **input_meta) as dest:
                dest.write(masked_data, 1)
    
    print(f"Masked raster saved to {output_tiff}")
    return output_tiff

# File paths
input_tiff = r"C:\Users\rubab\Desktop\GWE_codes\merged_data\transitionData.tif"
mask_tiff = r"C:\Users\rubab\Desktop\GWE_codes\Mask_CONUS_SentbyLaura\CONUS2_0_Final250m_Mask.tif"
output_tiff = r"C:\Users\rubab\Desktop\GWE_codes\merged_data\masked_output.tif"

# Perform masking
masked_file = mask_raster(input_tiff, mask_tiff, output_tiff)

# Display the masked output
with rasterio.open(masked_file) as src:
    show(src)


ValueError: Input and mask TIFFs must have the same dimensions.

In [12]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np
from rasterio.plot import show

def align_raster(src_path, dst_path, reference_path):
    """
    Align the source raster to the resolution, extent, and CRS of the reference raster.
    """
    with rasterio.open(reference_path) as ref:
        ref_meta = ref.meta.copy()
        
        with rasterio.open(src_path) as src:
            transform, width, height = calculate_default_transform(
                src.crs, ref.crs, ref.width, ref.height, *ref.bounds
            )
            ref_meta.update({
                "crs": ref.crs,
                "transform": ref.transform,
                "width": ref.width,
                "height": ref.height
            })
            
            with rasterio.open(dst_path, "w", **ref_meta) as dst:
                reproject(
                    source=rasterio.band(src, 1),
                    destination=rasterio.band(dst, 1),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=ref.transform,
                    dst_crs=ref.crs,
                    resampling=Resampling.nearest
                )
    print(f"Aligned raster saved to {dst_path}")
    return dst_path

def mask_raster(input_tiff, mask_tiff, output_tiff):
    """
    Mask the input raster using the aligned mask raster.
    """
    # Open the input TIFF
    with rasterio.open(input_tiff) as src:
        input_data = src.read(1)  # Read the first band
        input_meta = src.meta.copy()
        
        # Open the mask TIFF
        with rasterio.open(mask_tiff) as mask_src:
            mask_data = mask_src.read(1)  # Read the first band of the mask
            
            # Ensure input and mask have the same shape
            if src.shape != mask_src.shape:
                raise ValueError("Input and mask TIFFs must have the same dimensions.")
            
            # Mask out regions: Set input_data to nodata where mask_data > 0
            masked_data = np.where(mask_data > 0, np.nan, input_data)
            
            # Update metadata for nodata
            input_meta.update({"nodata": np.nan})
            
            # Save the masked raster
            with rasterio.open(output_tiff, "w", **input_meta) as dest:
                dest.write(masked_data, 1)
    
    print(f"Masked raster saved to {output_tiff}")
    return output_tiff

# File paths
input_tiff = r"C:\Users\rubab\Desktop\GWE_codes\merged_data\transitionData.tif"
mask_tiff = r"C:\Users\rubab\Desktop\GWE_codes\Mask_CONUS_SentbyLaura\CONUS2_0_Final250m_Mask.tif"
aligned_mask_tiff = r"C:\Users\rubab\Desktop\GWE_codes\Mask_CONUS_SentbyLaura\aligned_mask.tif"
output_tiff = r"C:\Users\rubab\Desktop\GWE_codes\merged_data\masked_output.tif"

# Align the mask raster to the input raster
align_raster(mask_tiff, aligned_mask_tiff, input_tiff)

# Perform masking
masked_file = mask_raster(input_tiff, aligned_mask_tiff, output_tiff)

# Display the masked output
with rasterio.open(masked_file) as src:
    show(src)


Aligned raster saved to C:\Users\rubab\Desktop\GWE_codes\Mask_CONUS_SentbyLaura\aligned_mask.tif


MemoryError: Unable to allocate 71.5 GiB for an array with shape (80000, 120000) and data type float64

In [16]:
import rasterio
import numpy as np
from rasterio.windows import from_bounds, Window
from rasterio.mask import mask

def mask_and_crop_raster_chunked(input_tiff, mask_tiff, output_tiff, chunk_size=512):
    """
    Mask and crop the input raster using the mask raster, processing in chunks.
    """
    with rasterio.open(mask_tiff) as mask_src:
        # Get the bounds of the mask TIFF
        mask_bounds = mask_src.bounds

        # Read the mask data
        mask_data = mask_src.read(1)

        # Create a polygon from the mask extent
        mask_polygon = [
            {
                "type": "Polygon",
                "coordinates": [[
                    [mask_bounds.left, mask_bounds.top],
                    [mask_bounds.right, mask_bounds.top],
                    [mask_bounds.right, mask_bounds.bottom],
                    [mask_bounds.left, mask_bounds.bottom],
                    [mask_bounds.left, mask_bounds.top]
                ]]
            }
        ]

    with rasterio.open(input_tiff) as src:
        # Calculate window for the mask bounds
        mask_window = from_bounds(
            mask_bounds.left, mask_bounds.bottom, mask_bounds.right, mask_bounds.top, src.transform
        )
        mask_window = Window(
            col_off=int(mask_window.col_off),
            row_off=int(mask_window.row_off),
            width=int(mask_window.width),
            height=int(mask_window.height)
        )

        # Prepare output metadata
        output_meta = src.meta.copy()
        output_meta.update({
            "driver": "GTiff",
            "height": int(mask_window.height),
            "width": int(mask_window.width),
            "transform": src.window_transform(mask_window),
            "nodata": np.nan,
            "dtype": "float32"
        })

        with rasterio.open(output_tiff, "w", **output_meta) as dest:
            for i in range(0, int(mask_window.height), chunk_size):
                for j in range(0, int(mask_window.width), chunk_size):
                    # Define the chunk window
                    height = min(chunk_size, mask_window.height - i)
                    width = min(chunk_size, mask_window.width - j)
                    chunk_window = Window(
                        col_off=mask_window.col_off + j,
                        row_off=mask_window.row_off + i,
                        width=width,
                        height=height
                    )

                    # Read input and mask data for the current chunk
                    input_data = src.read(1, window=chunk_window).astype("float32")
                    mask_chunk = mask_data[i:i + height, j:j + width]

                    # Apply masking
                    masked_chunk = np.where(mask_chunk > 0, np.nan, input_data)

                    # Write the masked chunk to the output raster
                    dest.write(masked_chunk, 1, window=chunk_window)

    print(f"Masked and cropped raster saved to {output_tiff}")
    return output_tiff

# File paths
input_tiff = r"C:\Users\rubab\Desktop\GWE_codes\merged_data\transitionData.tif"
mask_tiff = r"C:\Users\rubab\Desktop\GWE_codes\Mask_CONUS_SentbyLaura\CONUS2_0_Final250m_Mask.tif"
output_tiff = r"C:\Users\rubab\Desktop\GWE_codes\merged_data\cropped_masked_output.tif"

# Perform chunked masking and cropping
masked_file = mask_and_crop_raster_chunked(input_tiff, mask_tiff, output_tiff, chunk_size=512)


OverflowError: Python int too large to convert to C long

In [17]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np

def align_mask_to_input(mask_tiff, input_tiff, aligned_mask_tiff):
    """
    Align the mask raster to have the same dimensions, CRS, and resolution as the input raster.
    """
    with rasterio.open(input_tiff) as input_src:
        input_meta = input_src.meta.copy()

        with rasterio.open(mask_tiff) as mask_src:
            # Calculate the transform and dimensions to match the input raster
            transform, width, height = calculate_default_transform(
                mask_src.crs, input_src.crs, input_src.width, input_src.height, *input_src.bounds
            )
            aligned_meta = mask_src.meta.copy()
            aligned_meta.update({
                "crs": input_src.crs,
                "transform": input_src.transform,
                "width": input_src.width,
                "height": input_src.height
            })

            with rasterio.open(aligned_mask_tiff, "w", **aligned_meta) as aligned_dst:
                reproject(
                    source=rasterio.band(mask_src, 1),
                    destination=rasterio.band(aligned_dst, 1),
                    src_transform=mask_src.transform,
                    src_crs=mask_src.crs,
                    dst_transform=input_src.transform,
                    dst_crs=input_src.crs,
                    resampling=Resampling.nearest
                )
    print(f"Aligned mask saved to {aligned_mask_tiff}")
    return aligned_mask_tiff


def mask_raster(input_tiff, aligned_mask_tiff, output_tiff):
    """
    Mask the input raster using the aligned mask raster.
    """
    with rasterio.open(input_tiff) as src:
        input_data = src.read(1).astype("float32")  # Ensure compatibility with NaN
        input_meta = src.meta.copy()

        with rasterio.open(aligned_mask_tiff) as mask_src:
            mask_data = mask_src.read(1)

            # Perform masking: Set input_data to NaN where mask_data > 0
            masked_data = np.where(mask_data > 0, np.nan, input_data)

            # Update metadata for nodata and save the masked raster
            input_meta.update({"nodata": np.nan, "dtype": "float32"})
            with rasterio.open(output_tiff, "w", **input_meta) as dest:
                dest.write(masked_data, 1)

    print(f"Masked raster saved to {output_tiff}")
    return output_tiff


# File paths
input_tiff = r"C:\Users\rubab\Desktop\GWE_codes\merged_data\transitionData.tif"
mask_tiff = r"C:\Users\rubab\Desktop\GWE_codes\Mask_CONUS_SentbyLaura\CONUS2_0_Final250m_Mask.tif"
aligned_mask_tiff = r"C:\Users\rubab\Desktop\GWE_codes\Mask_CONUS_SentbyLaura\aligned_mask_final.tif"
output_tiff = r"C:\Users\rubab\Desktop\GWE_codes\merged_data\final_masked_output.tif"

# Align the mask to the input raster
aligned_mask = align_mask_to_input(mask_tiff, input_tiff, aligned_mask_tiff)

# Perform masking using the aligned mask
masked_file = mask_raster(input_tiff, aligned_mask, output_tiff)


Aligned mask saved to C:\Users\rubab\Desktop\GWE_codes\Mask_CONUS_SentbyLaura\aligned_mask_final.tif


MemoryError: Unable to allocate 35.8 GiB for an array with shape (1, 80000, 120000) and data type float32

In [18]:
import rasterio
import numpy as np
from rasterio.windows import Window

def mask_raster_chunked(input_tiff, aligned_mask_tiff, output_tiff, chunk_size=512):
    """
    Perform masking in chunks to avoid memory overload.
    """
    with rasterio.open(input_tiff) as src:
        input_meta = src.meta.copy()

        with rasterio.open(aligned_mask_tiff) as mask_src:
            # Ensure input and mask have the same dimensions
            if src.shape != mask_src.shape:
                raise ValueError("Input and mask TIFFs must have the same dimensions.")

            # Update metadata for the output
            input_meta.update({"nodata": np.nan, "dtype": "float32"})

            with rasterio.open(output_tiff, "w", **input_meta) as dest:
                for i in range(0, src.height, chunk_size):
                    for j in range(0, src.width, chunk_size):
                        # Define window dimensions
                        height = min(chunk_size, src.height - i)
                        width = min(chunk_size, src.width - j)
                        window = Window(j, i, width, height)

                        # Read input and mask data for the current window
                        input_data = src.read(1, window=window).astype("float32")
                        mask_data = mask_src.read(1, window=window)

                        # Apply masking
                        masked_chunk = np.where(mask_data > 0, np.nan, input_data)

                        # Write the masked chunk to the output raster
                        dest.write(masked_chunk, 1, window=window)

    print(f"Masked raster saved to {output_tiff}")
    return output_tiff

# File paths
input_tiff = r"C:\Users\rubab\Desktop\GWE_codes\merged_data\transitionData.tif"
aligned_mask_tiff = r"C:\Users\rubab\Desktop\GWE_codes\Mask_CONUS_SentbyLaura\aligned_mask_final.tif"
output_tiff = r"C:\Users\rubab\Desktop\GWE_codes\merged_data\final_masked_output_chunked.tif"

# Perform chunked masking
masked_file = mask_raster_chunked(input_tiff, aligned_mask_tiff, output_tiff, chunk_size=512)


Masked raster saved to C:\Users\rubab\Desktop\GWE_codes\merged_data\final_masked_output_chunked.tif


In [22]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

def reproject_raster(src_path, dst_path, target_crs):
    """
    Reproject a raster to a target CRS.
    """
    with rasterio.open(src_path) as src:
        transform, width, height = calculate_default_transform(
            src.crs, target_crs, src.width, src.height, *src.bounds
        )
        meta = src.meta.copy()
        meta.update({
            "crs": target_crs,
            "transform": transform,
            "width": width,
            "height": height
        })

        with rasterio.open(dst_path, "w", **meta) as dst:
            reproject(
                source=rasterio.band(src, 1),
                destination=rasterio.band(dst, 1),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=target_crs,
                resampling=Resampling.nearest
            )
    print(f"Reprojected raster saved to {dst_path}")
    return dst_path

# File paths
mask_tiff = r"C:\Users\rubab\Desktop\GWE_codes\Mask_CONUS_SentbyLaura\CONUS2_0_Final250m_Mask.tif"
input_tiff = r"C:\Users\rubab\Desktop\GWE_codes\merged_data\transitionData.tif"
reprojected_mask_tiff = r"C:\Users\rubab\Desktop\GWE_codes\Mask_CONUS_SentbyLaura\reprojected_mask.tif"

# Get the CRS of the input raster
with rasterio.open(input_tiff) as input_src:
    input_crs = input_src.crs

# Reproject the mask raster to the input raster's CRS
reprojected_mask = reproject_raster(mask_tiff, reprojected_mask_tiff, input_crs)


Reprojected raster saved to C:\Users\rubab\Desktop\GWE_codes\Mask_CONUS_SentbyLaura\reprojected_mask.tif


In [27]:
import rasterio
from rasterio.warp import reproject, Resampling

def clip_to_exact_extent(input_tiff, mask_tiff, output_tiff):
    """
    Clip the input raster to match the exact extent, resolution, and grid of the mask raster.
    """
    with rasterio.open(mask_tiff) as mask_src:
        # Get mask raster properties
        mask_bounds = mask_src.bounds
        mask_transform = mask_src.transform
        mask_crs = mask_src.crs
        mask_width = mask_src.width
        mask_height = mask_src.height

    with rasterio.open(input_tiff) as input_src:
        # Prepare output metadata
        output_meta = input_src.meta.copy()
        output_meta.update({
            "crs": mask_crs,
            "transform": mask_transform,
            "width": mask_width,
            "height": mask_height,
            "nodata": input_src.nodata or -9999,  # Default nodata value if none provided
            "dtype": "float32"  # Ensure compatible data type
        })

        # Create the output raster
        with rasterio.open(output_tiff, "w", **output_meta) as dest:
            # Reproject and clip the input raster to the mask raster's extent and grid
            reproject(
                source=rasterio.band(input_src, 1),
                destination=rasterio.band(dest, 1),
                src_transform=input_src.transform,
                src_crs=input_src.crs,
                dst_transform=mask_transform,
                dst_crs=mask_crs,
                dst_width=mask_width,
                dst_height=mask_height,
                resampling=Resampling.nearest
            )

    print(f"Clipped raster saved to {output_tiff}")
    return output_tiff

# File paths
input_tiff = r"C:\Users\rubab\Desktop\GWE_codes\merged_data\transitionData.tif"
reprojected_mask_tiff = r"C:\Users\rubab\Desktop\GWE_codes\Mask_CONUS_SentbyLaura\reprojected_mask.tif"
output_tiff = r"C:\Users\rubab\Desktop\GWE_codes\merged_data\clipped_to_reprojected_mask_exact.tif"

# Clip the input raster to the exact extent and resolution of the mask raster
clipped_file = clip_to_exact_extent(input_tiff, reprojected_mask_tiff, output_tiff)


Clipped raster saved to C:\Users\rubab\Desktop\GWE_codes\merged_data\clipped_to_reprojected_mask_exact.tif
